# Present Proof - Alice
## Role: Verifier

## This notebook works through the present proof protocol from the Verifier's perspective, it should be run alongside the prover notebook from Bob's perspective. 

## Before running through these two notebooks you should have run through Part 5 - Issue Credential. This can be found in these two notebooks: [Alice](http://localhost:8888/notebooks/Part%205%20-%20Issue%20Credential.ipynb), [Bob](http://localhost:8889/notebooks/Part%205%20-%20Issue%20Credential.ipynb)


If unfamiliar with the present-proof protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0037-present-proof)


## 1. Instantiate Controller for Alice's Agent

In [ ]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, api_key=API_KEY)



## 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic present_proof, printing out the payload. The agent calls to this webhook every time it receives a present proof protocol message from another agent. 

The code for this protocol can be found [here](https://github.com/hyperledger/aries-cloudagent-python/tree/master/aries_cloudagent/protocols/issue_credential).

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}
agent_controller.register_listeners([proof_listener], defaults=True)


## 3. Check the agent has an active connection

**Note: An active connection is required, this should have been established on start up through the python script create_connection.py in the setup folder. If not it is possible to run through the did-exchange tutorial to create one between Alice and Bob**

* [Alice](http://localhost:8888/notebooks/did-exchange-inviter.ipynb)
* [Bob](http://localhost:8889/notebooks/did-exchange-invitee.ipynb)

In [ ]:
try:
    response = await agent_controller.connections.get_connections()
    results = response['results']
    print("Results : ", results)
    print('\n')
    if len(results) > 0:
        connection = response['results'][0]
        print("Connection :", connection)
        if connection['state'] == 'active':
            connection_id = connection["connection_id"]
            print("\nActive Connection ID : ", connection_id)
        else:
            print("\nNo active connection found - wait a bit and execute again")
    else:
        print("You must create a connection")
except ConnectionRefusedError as e:
    print(repr(e))

## 4. Continue in [prover](http://127.0.0.1:8889/notebooks/Part%206%20-%20Present%20Proof.ipynb) Notebook

## 7. Send Proof Request

Request a proof of Opemined community member to prove community membership.



### 7.1. First we build the request object, a json. In the future we aim to develop a builder class that abstracts this away.

### TODO add better documentation to this whole process. Design for someome who has no idea what a proof request is. Hopefully the builder class will make that easier?

### Easy step for now would be to split up the sections and document each bit. Like the req_attrs. and req_preds. What restrictions are available etx.

In [ ]:
response = await agent_controller.wallet.get_public_did()
print(response)
issuer_did = response["result"]["did"]

print("Request proof of Name and Age range from Bob")
#Set some variables

revocation = True
SELF_ATTESTED = True
exchange_tracing = False

#Enable this to ask for attributes to identity a user
req_attrs = [
    {"name": "name", "restrictions": [{"issuer_did": issuer_did}]},
    {"name": "skill", "restrictions": [{"issuer_did": issuer_did}]},
]

if revocation:
    req_attrs.append(
        {
            "name": "skill",
            "restrictions": [{"issuer_did": issuer_did}],
            "non_revoked": {"to": int(time.time() - 1)},
        },
    )

if SELF_ATTESTED:
    # test self-attested claims
    req_attrs.append({"name": "country"},)

#Set predicates for Zero Knowledge Proofs
req_preds = [
    # test zero-knowledge proofs
    {
        "name": "age",
        "p_type": ">=",
        "p_value": 21,
        "restrictions": [{"issuer_did": issuer_did}],
    }
]

indy_proof_request = {
    "name": "Proof of Personal Information",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid":
        req_pred for req_pred in req_preds
    },
}

if revocation:
    indy_proof_request["non_revoked"] = {"to": int(time.time())}

#proof_request = indy_proof_request
exchange_tracing_id = exchange_tracing
proof_request_web_request = {
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    "trace": exchange_tracing,
}

{'result': {'did': 'PQRXDxdGqQGSZ8z69p4xZP', 'verkey': 'DDEKJtBjzaXAJtpwetdPiXH5s4rNUEzG18V15QoLRcZZ', 'posture': 'public'}}
Request proof of Name and Age range from Bob


In [5]:
proof_request_web_request

{'connection_id': 'a6f018df-8dd2-4044-b27c-001d3fabb1e5',
 'proof_request': {'name': 'Proof of Personal Information',
  'version': '1.0',
  'requested_attributes': {'0_name_uuid': {'name': 'name',
    'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]},
   '0_skill_uuid': {'name': 'skill',
    'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}],
    'non_revoked': {'to': 1606805041}},
   '0_country_uuid': {'name': 'country'}},
  'requested_predicates': {'0_age_GE_uuid': {'name': 'age',
    'p_type': '>=',
    'p_value': 21,
    'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}},
  'non_revoked': {'to': 1606805042}},
 'trace': False}

### 7.2. Send the proof request to Bob

Bob is identified through the connection_id

In [ ]:
response = await agent_controller.proofs.send_request(proof_request_web_request)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)


{'created_at': '2020-12-01 06:45:03.941578Z', 'state': 'request_sent', 'auto_present': False, 'role': 'verifier', 'connection_id': 'a6f018df-8dd2-4044-b27c-001d3fabb1e5', 'presentation_request': {'name': 'Proof of Personal Information', 'version': '1.0', 'requested_attributes': {'0_name_uuid': {'name': 'name', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_skill_uuid': {'name': 'skill', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}], 'non_revoked': {'to': 1606805041}}, '0_country_uuid': {'name': 'country'}}, 'requested_predicates': {'0_age_GE_uuid': {'name': 'age', 'p_type': '>=', 'p_value': 21, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'non_revoked': {'to': 1606805042}, 'nonce': '169094683509506946284602'}, 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': 'fbf67691-2b75-4bf6-bb52-1997675398c4', 'request_presentations~attach': [{'@id': 'libindy-request-presentatio

## 8. Continue in the prover notebook

## 11. Verify Proof Presentation

This is checking the signatures on the credentials presented against the credential schema and definition id stored and resolvable on the ledger.

### TODO: It is not clear to me how we actually take this big blob of data and parse it to make a meaningful, true false. Yes this proof is valid, no it is not. I think this is perhaps a event that gets fired to the handler? Additionally it would be good to show how you transform this to get the attributes that were proven. Should the controller provide some helper functions for this? Not sure, think it is more the role of the business logic using the controller. But would still be useful to show in the notebooks.

In [ ]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
print(verify)


{'created_at': '2020-11-30 05:55:04.777771Z', 'presentation': {'proof': {'proofs': [{'primary_proof': {'eq_proof': {'revealed_attrs': {'name': '93006290325627508022776103386395994712401809437930957652111221015872244345185', 'skill': '24214165878429772374367858032287880675266648921286067519661391650938186682695'}, 'a_prime': '56179246397531001242162228385021578622360324987525981041322320477568031048248320435904659012146257143251857640368769647103090392288761491607390840771184441198354753350662090019434077311419967547927668908605901250800914750121961599890482998408782446238982134543105267589135314117311884277481835526115273690941525261151829385089928756842979173486514955516582692210181867688377716835284151286248216399101905593900449225356458344258837228244940062994794313573854501204543195656461985186844712208352876733349480334035121079491171917576075320330380496504416363024039433707386540935661559811114491930424780157335536416979828', 'e': '73604627107865276737566220259762290291899290457

In [ ]:
print(verify['state'])
print(verify['state'] == 'verified')


verified
True


In [ ]:

# print(verify['presentation'])
# print(verify['presentation']['requested_proof'])

for (name, val) in verify['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print(name + " : " + val['raw'])


0_name_uuid : Bob
0_skill_uuid : researcher


In [ ]:

for (name, val) in verify['presentation']['requested_proof']['self_attested_attrs'].items():
    print(name + " : " + val)


0_country_uuid : South Africa


## Not all the proofs api has been covered in this example.

### TODO: At least list API calls not documented.

## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [ ]:
response = await agent_controller.terminate()
print(response)